# 🍿 Project: Hybrid Anime Recommender System
### *Bridging Collaborative Filtering & Semantic Search*

### **The Challenge**
Building a recommendation system is often a trade-off.
* **Collaborative Filtering** is great at finding patterns (*"People who liked X also liked Y"*), but it fails with new items (Cold Start) and doesn't understand content.
* **Content-Based Filtering** understands metadata, but often fails to capture user nuance.

### **The Solution: A Hybrid Architecture**
In this notebook, I am building a dual-engine system that combines the best of both worlds:
1.  **The "User Expert" (Neural Network):** A Deep Learning model (Neural Collaborative Filtering) that learns to predict user ratings based on interaction history.
2.  **The "Content Expert" (BERT):** A Transformer-based model that reads anime plot summaries to understand semantic similarity, powering a "Smart Search" feature.

In [ ]:
# First, I need to set up the environment.
# I'll be using TensorFlow for the Neural Network, Sentence-Transformers for BERT,
# and FAISS for high-performance vector search.

import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import faiss
import kagglehub
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

print("✅ Environment is set up and ready to go.")

## **Phase 1: Data Acquisition & Preprocessing**

High-quality data is the foundation of any good model. I am using the **Anime Recommendation Database 2020**, which contains millions of user ratings and detailed metadata.

**My Strategy:**
* **Data Ingestion:** Download fresh data directly via API.
* **Normalization:** The dataset uses non-standard column names (like `MAL_ID`), so I'll standardize everything to `anime_id`.
* **Noise Reduction:** To ensure the model learns meaningful patterns, I will filter out "sparse" data—users who have rated fewer than 50 anime, and anime that have fewer than 50 ratings. This removes noise and stabilizes the training process.

In [ ]:
# I'll grab the dataset directly from Kaggle to ensure reproducibility.
print("⬇️ Downloading Dataset...")
path = kagglehub.dataset_download("hernan4444/anime-recommendation-database-2020")

# Loading the specific files I need: Metadata and User Ratings.
anime_df = pd.read_csv(os.path.join(path, 'anime_with_synopsis.csv'))
rating_df = pd.read_csv(os.path.join(path, 'rating_complete.csv'))

# The dataset uses 'MAL_ID', but I prefer 'anime_id' for clarity in my codebase.
anime_df = anime_df.rename(columns={'MAL_ID': 'anime_id'})
print("✅ Column names standardized.")

# --- Filtering Strategy ---
# I want to train on 'active' users and 'known' items to avoid the cold-start noise.
min_ratings = 50

# Counting interactions
user_counts = rating_df['user_id'].value_counts()
anime_counts = rating_df['anime_id'].value_counts()

# Identifying valid candidates
valid_users = user_counts[user_counts >= min_ratings].index
valid_animes = anime_counts[anime_counts >= min_ratings].index

# Filtering the dataframes
df_filtered = rating_df[
    (rating_df['user_id'].isin(valid_users)) &
    (rating_df['anime_id'].isin(valid_animes))
].copy()

# Syncing the metadata to match the filtered ratings
anime_df = anime_df[anime_df['anime_id'].isin(valid_animes)].copy()
anime_df = anime_df.reset_index(drop=True)

print(f"✅ Data Preprocessing Complete.")
print(f"   Final Dataset: {len(anime_df)} Unique Animes, {len(df_filtered)} User Ratings.")

## **Phase 2: The "Collaborative Brain" (Neural Network)**

Now I will build the **Neural Collaborative Filtering (NCF)** model.

Instead of traditional Matrix Factorization, I am using a Deep Neural Network. This allows the model to learn non-linear relationships between users and items.

**Architecture:**
1.  **Input:** User ID and Anime ID.
2.  **Embeddings:** These IDs are passed through Embedding Layers, which convert them into dense vectors (latent features).
3.  **Concatenation:** I combine the User Vector and Item Vector.
4.  **Dense Layers:** The combined vector passes through hidden layers to extract complex patterns.
5.  **Output:** A single neuron with a `Sigmoid` activation, predicting a probability (0 to 1) of how much the user will like the item.

In [ ]:
print("🧠 Initializing Neural Collaborative Filtering (NCF) training...")

# 1. Encoding IDs
# Neural networks require contiguous integers (0, 1, 2...), not random IDs.
# So, I'll use LabelEncoder to map the real IDs to a sequence.
user_enc = LabelEncoder()
item_enc = LabelEncoder()

df_filtered['user'] = user_enc.fit_transform(df_filtered['user_id'])
df_filtered['item'] = item_enc.fit_transform(df_filtered['anime_id'])

num_users = df_filtered['user'].nunique()
num_items = df_filtered['item'].nunique()

# 2. Defining the Model Architecture
embedding_size = 32

# Input Layers
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

# Embedding Layers (The Latent Space)
user_embedding = Embedding(num_users, embedding_size, name='user_embedding')(user_input)
item_embedding = Embedding(num_items, embedding_size, name='item_embedding')(item_input)

# Flattening vectors to prepare for concatenation
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

# The Deep Learning part: Concatenate -> Dense Layers
concat = Concatenate()([user_vec, item_vec])
fc1 = Dense(128, activation='relu')(concat)
fc2 = Dense(64, activation='relu')(fc1)
output = Dense(1, activation='sigmoid')(fc2) # Output layer (Probability 0-1)

# Compiling the model
model = Model([user_input, item_input], output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Training
# I'm normalizing ratings to a 0-1 range to match the Sigmoid output.
y_train = (df_filtered['rating'] - 1) / 9.0

history = model.fit(
    [df_filtered['user'], df_filtered['item']],
    y_train,
    batch_size=2048, # Large batch size for faster GPU processing
    epochs=5,
    validation_split=0.1
)

# 4. Exporting the Model
# I'm saving this in the .h5 format to ensure compatibility with my backend.
model.save('anime_neumf_model.h5')
print("✅ Collaborative Model successfully trained and exported.")

## **Phase 3: The "Semantic Brain" (BERT & FAISS)**

The Neural Network handles *users*, but it doesn't understand what an anime actually *is*. For that, I need **Natural Language Processing (NLP)**.

**The Approach:**
1.  **Soup Creation:** I will combine the Name, Genres, and Synopsis into a single text string (a "soup") for every anime.
2.  **BERT Encoding:** I'll use `all-MiniLM-L6-v2`, a lightweight but powerful Transformer, to convert this text into a 384-dimensional vector.
3.  **Indexing:** I'll store these vectors in a **FAISS Index**. This allows my application to perform sub-millisecond similarity searches (e.g., finding anime with similar plots).

In [ ]:
print("🚀 Launching BERT for Semantic Analysis...")

# 1. Loading the Pre-trained Transformer
bert = SentenceTransformer('all-MiniLM-L6-v2')

# Handling inconsistent column naming in the source dataset
if 'sypnopsis' in anime_df.columns:
    anime_df = anime_df.rename(columns={'sypnopsis': 'Synopsis'})
elif 'synopsis' in anime_df.columns:
    anime_df = anime_df.rename(columns={'synopsis': 'Synopsis'})

# 2. Creating the 'Soup'
# This combines all relevant text metadata into one string for BERT to read.
anime_df['Genres'] = anime_df['Genres'].fillna('')
anime_df['Synopsis'] = anime_df['Synopsis'].fillna('')
anime_df['soup'] = anime_df['Name'] + ". " + anime_df['Genres'] + ". " + anime_df['Synopsis']

# 3. Vectorization
# This is the heavy lifting: converting text to numbers.
print("⏳ Generating Vector Embeddings...")
embeddings = bert.encode(
    anime_df['soup'].tolist(),
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

# 4. Building the FAISS Index
# Normalizing L2 allows us to use Cosine Similarity logic with efficient Inner Product search.
faiss.normalize_L2(embeddings)
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embeddings)

# Exporting the Index
faiss.write_index(index, "anime_vector_db.index")

# 5. Packaging Metadata for Production
# I need to save the Encoders and a lightweight dataframe so the API
# can interpret the model's output (Mapping ID -> Name).
indices = pd.Series(anime_df.index, index=anime_df['Name']).drop_duplicates().to_dict()

metadata = {
    'anime_df': anime_df[['anime_id', 'Name', 'Genres']],
    'indices': indices,
    'user_enc': user_enc,
    'item_enc': item_enc
}

with open('production_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print("✅ System Build Complete.")
print("   - Semantic Index saved: anime_vector_db.index")
print("   - Metadata Package saved: production_metadata.pkl")

## **Conclusion**
The training pipeline is complete. We have successfully generated the three core artifacts required for the backend API:

1.  `anime_neumf_model.h5`: The **Neural Network** that predicts user preference.
2.  `anime_vector_db.index`: The **FAISS Index** that powers the semantic search.
3.  `production_metadata.pkl`: The **Reference Data** (encoders and mappings).

These files are now ready to be moved to the `/backend` directory for deployment.